In [1]:
from with_FEM_MassMatrix import *
from convergence import *
from Reference_Solvers import *
from Experimental_Order_Cvg import *
from plots import *
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import csv
import pandas as pd
import plotly.express as px

In [2]:
alpha = 0.1
beta = 1.
NSamples =200
pList = [0.01, 0.02, 0.03, 0.04,0.05]
model ={'name': 'check', 'alpha': alpha, 'beta': beta}
NCoarse = np.array([4])
Nepsilon = np.array([128])
NFine = np.array([256])    # Number of "epsilon-blocks" on τ_ε mesh in each direction (1-D array: [x_ε, y_ε, z_ε] if 3D etc.)     # Number of "coarse-blocks" on τ_H mesh in each direction (1-D array: [x_H, y_H, z_H]) # Number of layers in the patch 
Neigen = 3
k =3
np.random.seed(123)
root = 'Data/LOD_1d_randcheck/N200_Ppt01_pt1_5_K3/_meanErr_H32'


mat = sio.loadmat(root)

mat = {k:v for k, v in mat.items() if k[0] != '_'}
data = pd.DataFrame({k: pd.Series(v[0]) for k, v in mat.items()}) # compatible for both python 2.x and python 3.x

data.to_csv("example.csv")


In [3]:
with open("example.csv", 'r') as file:
  csvreader = csv.reader(file)
  for row in csvreader:
    print(row)

['', 'absErr_1', 'absErr_2', 'pList']
['0', '9.284806266052215e-05', '0.0002782964746161909', '0.01']
['1', '0.0002650948725784197', '0.00010418448289861715', '0.02']
['2', '0.00028298099797785525', '0.00015773992959378091', '0.03']
['3', '0.00025123793430870123', '1.6548007711669754e-05', '0.04']
['4', '0.0001611933209484384', '0.0010890720571286039', '0.05']
['5', '8.008871558518038e-05', '0.0011268442944087909', '']
['6', '0.0003967737982986108', '0.00063254042718075', '']
['7', '0.0003056380464796504', '0.0006834163467290821', '']
['8', '0.0010030494488422192', '0.0002882315784002998', '']
['9', '0.0006702251964973982', '0.0009080267294656608', '']
['10', '0.000256294080077879', '0.00025629407999749887', '']
['11', '0.0005611035028274536', '0.0006488334234822091', '']
['12', '0.001575266223749594', '0.0011464988081435479', '']
['13', '6.955560868115995e-05', '0.00023491229815864045', '']
['14', '0.00036981214232012505', '0.0005546959290398945', '']
['15', '0.00025629408006766496', 

In [3]:
df = pd.read_csv("example.csv")

#check the first 5 rows

df.head()

,Unnamed: 0,absErr_1,absErr_2,pList
0,0,0.000093,0.000278,0.01
1,1,0.000265,0.000104,0.02
2,2,0.000283,0.000158,0.03
3,3,0.000251,0.000017,0.04
4,4,0.000161,0.001089,0.05


In [4]:
df.describe()[['absErr_1', 'absErr_2']]

,absErr_1,absErr_2
count,200.000000,2.000000e+02
mean,0.038811,3.604567e-02
std,0.383738,3.552883e-01
min,0.000006,7.664298e-07
25%,0.000237,2.562941e-04
50%,0.000256,2.562941e-04
75%,0.000451,5.956333e-04
max,3.948296,3.631295e+00


In [6]:
#create a histogram

fig1 = px.histogram(df, x='absErr_1')
fig1.show()

In [7]:
fig2 = px.histogram(df, x='absErr_2')
fig2.show()

In [10]:
#create a box plot

fig3 = px.box(df, y='absErr_1')
fig4 = px.box(df, y='absErr_2')
fig3.show()
fig4.show()

In [12]:
fig5 = px.scatter(x=df['Unnamed: 0'], y=df['absErr_1'])
fig5.show()

In [13]:
fig6 = px.scatter(x=df['Unnamed: 0'], y=df['absErr_2'])
fig6.show()

In [5]:
#create a function to find outliers using IQR

def find_outliers_IQR(df):
   q1=df.quantile(0.25)
   q3=df.quantile(0.75)
   IQR=q3-q1
   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
   return outliers

In [6]:
outliers_l1 = find_outliers_IQR(df['absErr_1'])
print("number of outliers: "+ str(len(outliers_l1)))
print("max outlier value: "+ str(outliers_l1.max()))
print("min outlier value: "+ str(outliers_l1.min()))
outliers_l1

number of outliers: 15
max outlier value: 3.9482962377979742
min outlier value: 0.0008273350618259


8      0.001003
12     0.001575
46     0.000844
57     0.001166
72     0.000948
84     3.948296
90     0.002315
110    0.000861
142    3.743770
146    0.000844
156    0.001270
172    0.001166
177    0.001110
190    0.000827
194    0.001179
Name: absErr_1, dtype: float64

In [20]:
outliers_l2 = find_outliers_IQR(df['absErr_2'])
print("number of outliers: "+ str(len(outliers_l2)))
print("max outlier value: "+ str(outliers_l2.max()))
print("min outlier value: "+ str(outliers_l2.min()))
outliers_l2

number of outliers: 13
max outlier value: 3.631294835574368
min outlier value: 0.0011268442944087


5      0.001127
12     0.001146
84     3.631295
90     0.001767
100    0.001166
111    0.001262
142    3.491880
156    0.002131
177    0.001374
187    0.001169
190    0.001151
194    0.001441
199    0.001395
Name: absErr_2, dtype: float64

In [24]:
#outliers = find_outliers_IQR(df[['absErr_1','absErr_2']])
#outliers

In [25]:
# Drop outlier points
def drop_outliers_IQR(df):
   q1=df.quantile(0.25)
   q3=df.quantile(0.75)
   IQR=q3-q1
   not_outliers = df[~((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
   outliers_dropped = outliers.dropna().reset_index()
   return outliers_dropped

In [7]:
# Cap outlier points
#  upper limit = the mean + 3*standard deviations
upper_limit = df['absErr_1'].mean() + 3*df['absErr_1'].std()
print(upper_limit)
lower_limit = df['absErr_1'].mean() - 3*df['absErr_1'].std()
print(lower_limit)

1.1900232442106646
-1.1124020272108042


In [8]:
df['absErr_1'] = np.where(df['absErr_1'] > upper_limit,
   upper_limit,
   np.where(
       df['absErr_1'] < lower_limit,
       lower_limit,
       df['absErr_1']
   )
)

In [9]:
df.describe()[['absErr_1']]

,absErr_1
count,200.000000
mean,0.012251
std,0.118668
min,0.000006
25%,0.000237
50%,0.000256
75%,0.000451
max,1.190023


In [10]:
#Replace outliers using imputation as if they were missing values
def impute_outliers_IQR(df):
   q1=df.quantile(0.25)
   q3=df.quantile(0.75)
   IQR=q3-q1
   upper = df[~(df>(q3+1.5*IQR))].max()
   lower = df[~(df<(q1-1.5*IQR))].min()
   df = np.where(df > upper,
       df.mean(),
       np.where(
           df < lower,
           df.mean(),
           df
           )
       )
   return df

In [11]:
df['absErr_1'] = impute_outliers_IQR(df['absErr_1'])
df.describe()['absErr_1']

count    200.000000
mean       0.001194
std        0.003161
min        0.000006
25%        0.000237
50%        0.000256
75%        0.000451
max        0.012251
Name: absErr_1, dtype: float64

In [12]:
df

,Unnamed: 0,absErr_1,absErr_2,pList
0,0,0.000093,0.000278,0.01
1,1,0.000265,0.000104,0.02
2,2,0.000283,0.000158,0.03
3,3,0.000251,0.000017,0.04
4,4,0.000161,0.001089,0.05
...,...,...,...,...
195,195,0.000256,0.000256,NaN
196,196,0.000032,0.000253,NaN
197,197,0.000302,0.000693,NaN
198,198,0.000256,0.000256,NaN


In [13]:
df.describe()

,Unnamed: 0,absErr_1,absErr_2,pList
count,200.000000,200.000000,2.000000e+02,5.000000
mean,99.500000,0.001194,3.604567e-02,0.030000
std,57.879185,0.003161,3.552883e-01,0.015811
min,0.000000,0.000006,7.664298e-07,0.010000
25%,49.750000,0.000237,2.562941e-04,0.020000
50%,99.500000,0.000256,2.562941e-04,0.030000
75%,149.250000,0.000451,5.956333e-04,0.040000
max,199.000000,0.012251,3.631295e+00,0.050000


In [18]:

#mat = {k:v for k, v in df.items() if k[0] != '_'}
#data = pd.DataFrame({k: pd.Series(v[0]) for k, v in df.items()}) 
#data.to_mat("example1.mat")
Root ='Data/LOD_1d_randcheck/N200_Ppt01_pt1_5_K3/'
#sio.savemat(Root)


OutData = {} 

# convert DF to dictionary before loading to your dictionary
OutData['Obj'] = df.to_dict('list')

sio.savemat(Root + 'testmat.mat',OutData)